In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm
import math
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import pickle

import datetime
warnings.filterwarnings('ignore')

## Features engineering

#### drl, lopsh, hocky, namhoc
drl: Điểm rèn luyện mỗi kỳ


lopsh: mã lớp sinh hoạt


namhoc: năm học hiện tại


hocky: học kỳ hiện tại trong năm học

In [3]:
drl = pd.read_excel('diemrl.xlsx')
drl.head()

,Unnamed: 0,id,mssv,lopsh,hocky,namhoc,drl,ghichu
0,0,27.0,599DFFB8XPvAibaEXe879+AOg1gh8lJvChSN7o+V,ANTN2013,2.0,2013.0,59.0,NULL
1,1,28.0,FC77E598XPvAibaEXe879+AOg1gh8pb/q8KzJ2A3,ANTN2013,2.0,2013.0,91.0,NULL
2,2,29.0,51E1C3E0XPvAibaEXe+4hxKfaQWuhLp2zzmiiyRe,ANTN2013,2.0,2013.0,93.0,NULL
3,3,30.0,6920B9AAXPvAibaEXe83EtiN4MI2ns6NZhlUAIhE,ANTN2013,2.0,2013.0,69.0,NULL
4,4,31.0,2F237AA9XPvAibaEXe/YKAlYnC3m967dOM4WK2IJ,ANTN2013,2.0,2013.0,72.0,NULL


In [4]:
drl = drl.drop(columns=['Unnamed: 0'])
drl.isna().sum()

,0
id,34057
mssv,56
lopsh,56
hocky,56
namhoc,56
drl,56
ghichu,56


In [5]:
drl = drl.dropna(how='all')
drl = drl.drop_duplicates()

In [6]:
drl = drl.sort_values('mssv', kind='mergesort')
drl = drl.reset_index(drop=True)
drl.head()

,id,mssv,lopsh,hocky,namhoc,drl,ghichu
0,NaN,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,TMĐT2019,1.0,2019.0,100.0,'')
1,NaN,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,TMĐT2019,2.0,2019.0,100.0,'')
2,NaN,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,TMĐT2019,1.0,2020.0,100.0,'TMĐT2019')
3,1529.0,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,KTMT2013,2.0,2013.0,76.0,NULL
4,19710.0,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,KTMT2013,1.0,2014.0,75.0,NULL


In [7]:
with pd.option_context("display.max_rows", 1000):
  display(drl['lopsh'].value_counts())

,count
lopsh,
KTPM2013,1236
MMTT2014,1218
MMTT2013,1194
KTMT2014,1131
KTPM2014,1099
KHMT2016,1015
HTTT2014,1015
PMCL2016,1014
KHMT2013,1003


#### hedaotao
Derived Features, dựa vào mã lớp sinh hoạt

In [9]:
taiNang = ['ATTN', 'KHTN']
tienTien = ['CTTT']
clc = ['TMCL','PMCL', 'MTCL', 'MMCL','KHCL','HTCL','CNCL','ATCL']

def heDaoTao(lopsh):
  try:
    if (lopsh[1:5] in taiNang):
      return 3
    elif (lopsh[1:5] in tienTien):
      return 2
    elif (lopsh[1:5] in clc):
      return 1
    else:
      return 0
  except:
    return 0

drl['hedaotao'] = drl['lopsh'].map(heDaoTao)
drl.head()

,id,mssv,lopsh,hocky,namhoc,drl,ghichu,hedaotao
0,NaN,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,TMĐT2019,1.0,2019.0,100.0,''),0
1,NaN,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,TMĐT2019,2.0,2019.0,100.0,''),0
2,NaN,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,TMĐT2019,1.0,2020.0,100.0,'TMĐT2019'),0
3,1529.0,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,KTMT2013,2.0,2013.0,76.0,NULL,0
4,19710.0,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,KTMT2013,1.0,2014.0,75.0,NULL,0


#### diemtb
Derived feature, điểm trung bình tính đến thời điểm hiện tại

##### Tạo cột namnhaphoc (Năm nhập học)

Xử lí trường hợp sinh viên lớp KTMT0001

In [10]:
drl.loc[drl['lopsh'] == ' KTMT0001']

,id,mssv,lopsh,hocky,namhoc,drl,ghichu,hedaotao
12,NaN,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,KTMT0001,2.0,2017.0,70.0,NULL),0
197,NaN,00AD2952XPvAibaEXe/45+b0LBlJ7GBrgcGKBM3K,KTMT0001,2.0,2019.0,50.0,''),0
198,NaN,00AD2952XPvAibaEXe/45+b0LBlJ7GBrgcGKBM3K,KTMT0001,1.0,2020.0,53.0,'KTMT0001'),0
247,NaN,00DC581CXPvAibaEXe9KG2OR8ZVK3umWFpCcHAl6,KTMT0001,2.0,2019.0,50.0,''),0
248,NaN,00DC581CXPvAibaEXe9KG2OR8ZVK3umWFpCcHAl6,KTMT0001,1.0,2020.0,58.0,'KTMT0001'),0
...,...,...,...,...,...,...,...,...
53446,NaN,FDA9DEA1XPvAibaEXe+TQJLvJOj4Bbq7Gu36vTA9,KTMT0001,1.0,2020.0,53.0,'KTMT0001'),0
53645,NaN,FE692FB2XPvAibaEXe/Q6C3BndaQUMQ/+BspERWE,KTMT0001,2.0,2019.0,66.0,''),0
53646,NaN,FE692FB2XPvAibaEXe/Q6C3BndaQUMQ/+BspERWE,KTMT0001,1.0,2020.0,71.0,'KTMT0001'),0
53883,NaN,FF5BFE8AXPvAibaEXe9ysTifnGoMwSIhZIOZGH/s,KTMT0001,2.0,2017.0,75.0,NULL),0


In [11]:
for i in tqdm(range(len(drl))):

  if len(str(drl['lopsh'][i])) > 9:
    drl['lopsh'][i] = drl['lopsh'][i][:9]

  if ('0001' in str(drl['lopsh'][i])) | (drl['lopsh'][i] == ' '):
    # print(drl.mssv[i],drl.lopsh[i],drl.mssv[i-1])
    check = True
    goBack = 1
    while check:
      if drl['mssv'][i-goBack] == drl['mssv'][i]:
        if ('0001' not in drl['lopsh'][i-goBack]):
          drl['lopsh'][i] = drl['lopsh'][i-goBack]
          check = False
      else:
        check = False
      goBack += 1

with pd.option_context("display.max_rows", 1000):
  display(drl['lopsh'].value_counts())

100%|██████████| 54001/54001 [00:03<00:00, 16462.24it/s]


,count
lopsh,
KTPM2013,1673
MMTT2013,1518
MMTT2014,1317
KHMT2013,1294
KTMT2014,1233
KTPM2014,1221
HTTT2014,1132
KTMT2013,1081
PMCL2015,1060


In [12]:
dtb = pd.read_excel('sinhvien_dtb_hocky.xlsx')
dtb.head()

,mssv,hocky,namhoc,dtbhk,sotchk
0,48685401XPvAibaEXe99yNsueYx+scOnjQSx4nXO,1,2016,6.45,13
1,48685401XPvAibaEXe99yNsueYx+scOnjQSx4nXO,2,2016,5.86,17
2,48685401XPvAibaEXe99yNsueYx+scOnjQSx4nXO,1,2017,6.54,16
3,48685401XPvAibaEXe99yNsueYx+scOnjQSx4nXO,2,2017,5.67,23
4,48685401XPvAibaEXe99yNsueYx+scOnjQSx4nXO,1,2018,2.27,21


In [13]:
df = dtb.merge(drl[['mssv', 'lopsh','hocky','namhoc','hedaotao']], how='inner', on=['mssv','hocky','namhoc'])
df = df.sort_values('mssv', kind='mergesort')
df = df.reset_index(drop=True)
df.head()

,mssv,hocky,namhoc,dtbhk,sotchk,lopsh,hedaotao
0,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,1,2019,8.84,17,TMĐT2019,0
1,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,2,2019,9.00,21,TMĐT2019,0
2,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,1,2020,9.11,15,TMĐT2019,0
3,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,1,2013,5.41,22,KTMT2013,0
4,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,2,2013,7.07,15,KTMT2013,0


In [16]:
def namNhapHoc(lopsh):
  try:
    return int(lopsh[-4:])
  except:
    return None

df['namnhaphoc'] = df['lopsh'].map(namNhapHoc)
df.head()

,mssv,hocky,namhoc,dtbhk,sotchk,lopsh,hedaotao,namnhaphoc
0,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,1,2019,8.84,17,TMĐT2019,0,2019.0
1,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,2,2019,9.00,21,TMĐT2019,0,2019.0
2,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,1,2020,9.11,15,TMĐT2019,0,2019.0
3,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,1,2013,5.41,22,KTMT2013,0,2013.0
4,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,2,2013,7.07,15,KTMT2013,0,2013.0


##### Tạo cột hockythu (Học kỳ thứ so với thời điểm nhập học)

In [17]:
def cotHocKyThu(df):
  soKys = []
  for i in range(len(df)):
    soKy = (df['namhoc'][i] - df['namnhaphoc'][i])*2 + df['hocky'][i]
    soKys.append(soKy)
  # df['hoc_ky_thu'] = col
  return soKys

df['hockythu'] = cotHocKyThu(df)
df.head()

,mssv,hocky,namhoc,dtbhk,sotchk,lopsh,hedaotao,namnhaphoc,hockythu
0,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,1,2019,8.84,17,TMĐT2019,0,2019.0,1.0
1,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,2,2019,9.00,21,TMĐT2019,0,2019.0,2.0
2,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,1,2020,9.11,15,TMĐT2019,0,2019.0,3.0
3,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,1,2013,5.41,22,KTMT2013,0,2013.0,1.0
4,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,2,2013,7.07,15,KTMT2013,0,2013.0,2.0


##### Tính điểm tb tổng thể đến thời điểm hiện tại mỗi kì

In [18]:
def cotDiemTb(df):
  cdtb = []
  for i in range(len(df)):
    if df['hockythu'][i] == 1:
      cdtb.append(df['dtbhk'][i])
    else:
      j = i-1
      s = df['dtbhk'][i]
      while df['mssv'][j] == df['mssv'][i]:
        s += df['dtbhk'][j]
        j -= 1
        if j < 0:
          break
      cdtb.append(s/df['hockythu'][i])

  return cdtb

df['diemtb'] = cotDiemTb(df)
df.head()

,mssv,hocky,namhoc,dtbhk,sotchk,lopsh,hedaotao,namnhaphoc,hockythu,diemtb
0,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,1,2019,8.84,17,TMĐT2019,0,2019.0,1.0,8.840000
1,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,2,2019,9.00,21,TMĐT2019,0,2019.0,2.0,8.920000
2,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,1,2020,9.11,15,TMĐT2019,0,2019.0,3.0,8.983333
3,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,1,2013,5.41,22,KTMT2013,0,2013.0,1.0,5.410000
4,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,2,2013,7.07,15,KTMT2013,0,2013.0,2.0,6.240000


#### loaixn, listening, speaking, reading, writing
loaixn: Loại chứng chỉ ngoại ngữ

listening: Điểm thi chứng chỉ phần kỹ năng nghe

speaking: Điểm thi chứng chỉ phần kỹ năng nói

reading: Điểm thi chứng chỉ phần kỹ năng đọc

writing: Điểm thi chứng chỉ phần kỹ năng viết


In [20]:
cc = pd.read_excel('sinhvien_chungchi.xlsx')
cc = cc.dropna(how='all',subset=['listening','speaking','reading','writing','tongdiem'])
cc = cc[(cc['loaixn'] != 'NHAT') & (cc['loaixn'] != 'PHAP') & (cc['loaixn'] != 'DGNL')]
cc = cc.drop_duplicates(subset=['mssv','ngaythi','loaixn','listening','speaking','reading','writing','tongdiem'])
cc = cc.sort_values('mssv', kind='mergesort')
cc.head()

,Unnamed: 0,id,mssv,ngaythi,url,loaixn,listening,speaking,reading,writing,tongdiem,lydo,trangthai,ngayxl,column0,Column1,Column2
1007,1007,2802.0,005D6433XPvAibaEXe9JJ0DEqozv6JkD0BJUpbna,2020-07-18,bangcap/16521119/16521119_bangcap_VNU-EPT_2020...,VNU-EPT,NaN,NaN,NaN,NaN,262,NaN,1,NaN,NaT,NaN,NaN
347,347,547.0,006B0A45XPvAibaEXe+TZKE2gC8nUj96gTN+8Bxy,2011-07-26,bangcap/14520881/14520881_bangcap_TOEIC_LR_201...,TOEIC_LR,345.0,NaN,325.0,NaN,NaN,NaN,1,2019-04-12 15:11:33,NaT,NaN,NaN
348,348,548.0,006B0A45XPvAibaEXe+TZKE2gC8nUj96gTN+8Bxy,2011-07-26,bangcap/14520881/14520881_bangcap_TOEIC_SW_201...,TOEIC_SW,NaN,110.0,NaN,110.0,NaN,NaN,1,2019-10-22 10:55:05,NaT,NaN,NaN
702,702,988.0,00A65C83XPvAibaEXe+BUGeEgVnB4GtsCU49A5+8,2019-04-06,bangcap/15520978/15520978_bangcap_VNU-EPT_2019...,VNU-EPT,NaN,NaN,NaN,NaN,202,NaN,1,2020-01-03 14:12:42,NaT,NaN,NaN
1643,1643,4415.0,00AD2952XPvAibaEXe/45+b0LBlJ7GBrgcGKBM3K,2020-08-16,bangcap/15520386/15520386_bangcap_TOEIC_LR_202...,TOEIC_LR,280.0,NaN,230.0,NaN,510,NaN,1,2021-05-27 16:17:45,NaT,NaN,NaN


In [21]:
cc = cc.merge(df[['mssv','namnhaphoc']].drop_duplicates(), how='inner', on='mssv')
cc.head()

,Unnamed: 0,id,mssv,ngaythi,url,loaixn,listening,speaking,reading,writing,tongdiem,lydo,trangthai,ngayxl,column0,Column1,Column2,namnhaphoc
0,1007,2802.0,005D6433XPvAibaEXe9JJ0DEqozv6JkD0BJUpbna,2020-07-18,bangcap/16521119/16521119_bangcap_VNU-EPT_2020...,VNU-EPT,NaN,NaN,NaN,NaN,262,NaN,1,NaN,NaT,NaN,NaN,2016.0
1,347,547.0,006B0A45XPvAibaEXe+TZKE2gC8nUj96gTN+8Bxy,2011-07-26,bangcap/14520881/14520881_bangcap_TOEIC_LR_201...,TOEIC_LR,345.0,NaN,325.0,NaN,NaN,NaN,1,2019-04-12 15:11:33,NaT,NaN,NaN,2014.0
2,348,548.0,006B0A45XPvAibaEXe+TZKE2gC8nUj96gTN+8Bxy,2011-07-26,bangcap/14520881/14520881_bangcap_TOEIC_SW_201...,TOEIC_SW,NaN,110.0,NaN,110.0,NaN,NaN,1,2019-10-22 10:55:05,NaT,NaN,NaN,2014.0
3,702,988.0,00A65C83XPvAibaEXe+BUGeEgVnB4GtsCU49A5+8,2019-04-06,bangcap/15520978/15520978_bangcap_VNU-EPT_2019...,VNU-EPT,NaN,NaN,NaN,NaN,202,NaN,1,2020-01-03 14:12:42,NaT,NaN,NaN,2015.0
4,1643,4415.0,00AD2952XPvAibaEXe/45+b0LBlJ7GBrgcGKBM3K,2020-08-16,bangcap/15520386/15520386_bangcap_TOEIC_LR_202...,TOEIC_LR,280.0,NaN,230.0,NaN,510,NaN,1,2021-05-27 16:17:45,NaT,NaN,NaN,2015.0


Tính học kỳ thứ tại thời điểm nộp chứng chỉ

In [22]:
format1 = '%Y-%m-%d %H:%M:%S'
format2 = '%Y-%m-%d'

ma = max(df['hockythu'])
def hocKyThuCc(df):
  soKys = []
  for i in range(len(df)):
    ngayNhapHoc = datetime.datetime.strptime(str(int(df['namnhaphoc'][i])) + '-09-08', format2)
    try:
      ngayXl = datetime.datetime.strptime(str(df['ngayxl'][i].strip()), format1)
    except:
      try:
        ngayXl = datetime.datetime.strptime(str(df['ngaythi'][i].strip()), format2)
      except:
        ngayXl = ngayNhapHoc

    num = math.ceil((ngayXl - ngayNhapHoc).days / 182)
    if num < 1:
      num = 1
    else:
      if num > ma:
        num = ma
    soKys.append(num)

  return soKys

cc['hockythu']  = hocKyThuCc(cc)
cc.head()

,Unnamed: 0,id,mssv,ngaythi,url,loaixn,listening,speaking,reading,writing,tongdiem,lydo,trangthai,ngayxl,column0,Column1,Column2,namnhaphoc,hockythu
0,1007,2802.0,005D6433XPvAibaEXe9JJ0DEqozv6JkD0BJUpbna,2020-07-18,bangcap/16521119/16521119_bangcap_VNU-EPT_2020...,VNU-EPT,NaN,NaN,NaN,NaN,262,NaN,1,NaN,NaT,NaN,NaN,2016.0,8.0
1,347,547.0,006B0A45XPvAibaEXe+TZKE2gC8nUj96gTN+8Bxy,2011-07-26,bangcap/14520881/14520881_bangcap_TOEIC_LR_201...,TOEIC_LR,345.0,NaN,325.0,NaN,NaN,NaN,1,2019-04-12 15:11:33,NaT,NaN,NaN,2014.0,10.0
2,348,548.0,006B0A45XPvAibaEXe+TZKE2gC8nUj96gTN+8Bxy,2011-07-26,bangcap/14520881/14520881_bangcap_TOEIC_SW_201...,TOEIC_SW,NaN,110.0,NaN,110.0,NaN,NaN,1,2019-10-22 10:55:05,NaT,NaN,NaN,2014.0,11.0
3,702,988.0,00A65C83XPvAibaEXe+BUGeEgVnB4GtsCU49A5+8,2019-04-06,bangcap/15520978/15520978_bangcap_VNU-EPT_2019...,VNU-EPT,NaN,NaN,NaN,NaN,202,NaN,1,2020-01-03 14:12:42,NaT,NaN,NaN,2015.0,9.0
4,1643,4415.0,00AD2952XPvAibaEXe/45+b0LBlJ7GBrgcGKBM3K,2020-08-16,bangcap/15520386/15520386_bangcap_TOEIC_LR_202...,TOEIC_LR,280.0,NaN,230.0,NaN,510,NaN,1,2021-05-27 16:17:45,NaT,NaN,NaN,2015.0,12.0


Forward fill kết quả thi chứng chỉ cho các học kỳ sau

In [23]:
for i in tqdm(range(1,len(cc))):

  if str(cc['listening'][i])=='nan':
    # print(cc.mssv[i], cc['listening'][i],cc['listening'][i-1])
    check = True
    goBack = 1
    while check:
      if cc['mssv'][i-goBack] == cc['mssv'][i]:
        # print('here')
        if str(cc['listening'][i-goBack]) != 'nan':
          cc['listening'][i] = cc['listening'][i-goBack]
          cc['reading'][i] = cc['reading'][i-goBack]
          check = False
      else:
        check = False
      goBack += 1

  if str(cc['writing'][i])=='nan':
    check = True
    goBack = 1
    while check:
      if cc['mssv'][i-goBack] == cc['mssv'][i]:
        if str(cc['writing'][i-goBack]) != 'nan':
          cc['writing'][i] = cc['writing'][i-goBack]
          cc['speaking'][i] = cc['speaking'][i-goBack]
          check = False
      else:
        check = False
      goBack += 1

cc.head()

100%|██████████| 1451/1451 [00:00<00:00, 11397.93it/s]


,Unnamed: 0,id,mssv,ngaythi,url,loaixn,listening,speaking,reading,writing,tongdiem,lydo,trangthai,ngayxl,column0,Column1,Column2,namnhaphoc,hockythu
0,1007,2802.0,005D6433XPvAibaEXe9JJ0DEqozv6JkD0BJUpbna,2020-07-18,bangcap/16521119/16521119_bangcap_VNU-EPT_2020...,VNU-EPT,NaN,NaN,NaN,NaN,262,NaN,1,NaN,NaT,NaN,NaN,2016.0,8.0
1,347,547.0,006B0A45XPvAibaEXe+TZKE2gC8nUj96gTN+8Bxy,2011-07-26,bangcap/14520881/14520881_bangcap_TOEIC_LR_201...,TOEIC_LR,345.0,NaN,325.0,NaN,NaN,NaN,1,2019-04-12 15:11:33,NaT,NaN,NaN,2014.0,10.0
2,348,548.0,006B0A45XPvAibaEXe+TZKE2gC8nUj96gTN+8Bxy,2011-07-26,bangcap/14520881/14520881_bangcap_TOEIC_SW_201...,TOEIC_SW,345.0,110.0,325.0,110.0,NaN,NaN,1,2019-10-22 10:55:05,NaT,NaN,NaN,2014.0,11.0
3,702,988.0,00A65C83XPvAibaEXe+BUGeEgVnB4GtsCU49A5+8,2019-04-06,bangcap/15520978/15520978_bangcap_VNU-EPT_2019...,VNU-EPT,NaN,NaN,NaN,NaN,202,NaN,1,2020-01-03 14:12:42,NaT,NaN,NaN,2015.0,9.0
4,1643,4415.0,00AD2952XPvAibaEXe/45+b0LBlJ7GBrgcGKBM3K,2020-08-16,bangcap/15520386/15520386_bangcap_TOEIC_LR_202...,TOEIC_LR,280.0,NaN,230.0,NaN,510,NaN,1,2021-05-27 16:17:45,NaT,NaN,NaN,2015.0,12.0


In [24]:
cc['loaixn'].value_counts()

,count
loaixn,
TOEIC_LR,1141
TOEIC_SW,144
IELTS,90
VNU-EPT,68
Cambrigde,8
TOEFL iBT,1


Chuyển đổi thang điểm sang TOEIC

In [25]:
def vnuEptToToeic(x):
  x = int(x)
  if x < 75:
    lr, sw = 60, 40
  elif x < 100:
    lr, sw = 120, 80
  elif x < 125:
    lr, sw = 170, 120
  elif x < 150:
    lr,sw = 225, 160
  elif x < 175 :
    lr, sw = 315, 180
  elif x < 200:
    lr, sw = 400, 200
  elif x < 225:
    lr, sw = 475, 220
  elif x < 250:
    lr, sw = 550, 240
  elif x < 275:
    lr, sw = 670, 270
  elif x < 300:
    lr, sw = 785, 310
  elif x < 325:
    lr, sw = 885, 350
  elif x < 350:
    lr, sw = 945, 380
  else:
    lr, sw = 990, 400
  return int(lr/2), int(sw/2), int(lr/2), int(sw/2)

def ieltsToToeic(x):
  x = float(x)
  if x < 2.5:
    lr, sw = 120, 80
  elif x < 3:
    lr, sw = 170, 120
  elif x < 3.5:
    lr,sw = 225, 160
  elif x < 4 :
    lr, sw = 315, 180
  elif x < 4.5:
    lr, sw = 400, 200
  elif x < 5:
    lr, sw = 475, 220
  elif x < 5.5:
    lr, sw = 550, 240
  elif x < 6:
    lr, sw = 670, 270
  elif x < 6.5:
    lr, sw = 785, 310
  elif x < 7:
    lr, sw = 885, 350
  elif x < 7.5:
    lr, sw = 945, 380
  else:
    lr, sw = 990, 400
  return int(lr/2), int(sw/2), int(lr/2), int(sw/2)

def cambrigdeToToeic(x):
  x = int(x)
  if x < 120 :
    lr, sw = 225, 120
  elif x < 140:
    lr, sw = 400, 180
  elif x < 160:
    lr, sw = 450, 200
  elif x < 180:
    lr, sw = 600, 280
  elif x < 200:
    lr, sw = 850, 350
  elif x < 215:
    lr, sw = 910, 380
  else:
    lr, sw = 990, 400
  return int(lr/2), int(sw/2), int(lr/2), int(sw/2)

def toeflToToeic(x):
  x = int(x)
  if x < 31 :
    lr, sw = 315, 180
  elif x < 34:
    lr, sw = 400, 200
  elif x < 45:
    lr, sw = 475, 220
  elif x < 59:
    lr, sw = 550, 240
  elif x < 78:
    lr, sw = 670, 270
  elif x < 93:
    lr, sw = 785, 310
  elif x < 101:
    lr, sw = 885, 350
  elif x < 114:
    lr, sw = 945, 380
  else:
    lr, sw = 990, 400
  return int(lr/2), int(sw/2), int(lr/2), int(sw/2)


def convertChungChi(df):
  for i in range(df.shape[0]):
    if df.loaixn[i] == 'VNU-EPT':
      df.listening[i], df.speaking[i], df.reading[i], df.writing[i] = vnuEptToToeic(df.tongdiem[i])
    elif df.loaixn[i] == 'IELTS':
      df.listening[i], df.speaking[i], df.reading[i], df.writing[i] = ieltsToToeic(df.tongdiem[i])
    elif df.loaixn[i] == 'Cambrigde':
      df.listening[i], df.speaking[i], df.reading[i], df.writing[i] = cambrigdeToToeic(df.tongdiem[i])
    elif df.loaixn[i] == 'TOEFL iBT':
      df.listening[i], df.speaking[i], df.reading[i], df.writing[i] = toeflToToeic(df.tongdiem[i])
  return df

cc = convertChungChi(cc)
cc.head()

,Unnamed: 0,id,mssv,ngaythi,url,loaixn,listening,speaking,reading,writing,tongdiem,lydo,trangthai,ngayxl,column0,Column1,Column2,namnhaphoc,hockythu
0,1007,2802.0,005D6433XPvAibaEXe9JJ0DEqozv6JkD0BJUpbna,2020-07-18,bangcap/16521119/16521119_bangcap_VNU-EPT_2020...,VNU-EPT,335.0,135.0,335.0,135.0,262,NaN,1,NaN,NaT,NaN,NaN,2016.0,8.0
1,347,547.0,006B0A45XPvAibaEXe+TZKE2gC8nUj96gTN+8Bxy,2011-07-26,bangcap/14520881/14520881_bangcap_TOEIC_LR_201...,TOEIC_LR,345.0,NaN,325.0,NaN,NaN,NaN,1,2019-04-12 15:11:33,NaT,NaN,NaN,2014.0,10.0
2,348,548.0,006B0A45XPvAibaEXe+TZKE2gC8nUj96gTN+8Bxy,2011-07-26,bangcap/14520881/14520881_bangcap_TOEIC_SW_201...,TOEIC_SW,345.0,110.0,325.0,110.0,NaN,NaN,1,2019-10-22 10:55:05,NaT,NaN,NaN,2014.0,11.0
3,702,988.0,00A65C83XPvAibaEXe+BUGeEgVnB4GtsCU49A5+8,2019-04-06,bangcap/15520978/15520978_bangcap_VNU-EPT_2019...,VNU-EPT,237.0,110.0,237.0,110.0,202,NaN,1,2020-01-03 14:12:42,NaT,NaN,NaN,2015.0,9.0
4,1643,4415.0,00AD2952XPvAibaEXe/45+b0LBlJ7GBrgcGKBM3K,2020-08-16,bangcap/15520386/15520386_bangcap_TOEIC_LR_202...,TOEIC_LR,280.0,NaN,230.0,NaN,510,NaN,1,2021-05-27 16:17:45,NaT,NaN,NaN,2015.0,12.0


Fill các giá trị điểm thi chứng chỉ trống bằng giá trị trung bình ứng với số kỳ đã học

In [26]:
def tb(l):
  s = 0
  c = 0
  for i in l:
    if str(i) != 'nan':
      s += i
      c +=1
  return int(s/c)

avgs = {}
for i in range(1,14):
  for j in ['listening','speaking','reading','writing']:
    avgs[i,j] = tb(cc[cc.hockythu == i][j])

def fillEmptyScore(df):
  for i in range(df.shape[0]):
    for j in ['listening','speaking','reading','writing']:
      if str(df[j][i]) == 'nan':
        df[j][i] = avgs[df.hockythu[i],j]
  return df

cc = fillEmptyScore(cc)
cc

,Unnamed: 0,id,mssv,ngaythi,url,loaixn,listening,speaking,reading,writing,tongdiem,lydo,trangthai,ngayxl,column0,Column1,Column2,namnhaphoc,hockythu
0,1007,2802.0,005D6433XPvAibaEXe9JJ0DEqozv6JkD0BJUpbna,2020-07-18,bangcap/16521119/16521119_bangcap_VNU-EPT_2020...,VNU-EPT,335.0,135.0,335.0,135.0,262,NaN,1,NaN,NaT,NaN,NaN,2016.0,8.0
1,347,547.0,006B0A45XPvAibaEXe+TZKE2gC8nUj96gTN+8Bxy,2011-07-26,bangcap/14520881/14520881_bangcap_TOEIC_LR_201...,TOEIC_LR,345.0,122.0,325.0,124.0,NaN,NaN,1,2019-04-12 15:11:33,NaT,NaN,NaN,2014.0,10.0
2,348,548.0,006B0A45XPvAibaEXe+TZKE2gC8nUj96gTN+8Bxy,2011-07-26,bangcap/14520881/14520881_bangcap_TOEIC_SW_201...,TOEIC_SW,345.0,110.0,325.0,110.0,NaN,NaN,1,2019-10-22 10:55:05,NaT,NaN,NaN,2014.0,11.0
3,702,988.0,00A65C83XPvAibaEXe+BUGeEgVnB4GtsCU49A5+8,2019-04-06,bangcap/15520978/15520978_bangcap_VNU-EPT_2019...,VNU-EPT,237.0,110.0,237.0,110.0,202,NaN,1,2020-01-03 14:12:42,NaT,NaN,NaN,2015.0,9.0
4,1643,4415.0,00AD2952XPvAibaEXe/45+b0LBlJ7GBrgcGKBM3K,2020-08-16,bangcap/15520386/15520386_bangcap_TOEIC_LR_202...,TOEIC_LR,280.0,103.0,230.0,111.0,510,NaN,1,2021-05-27 16:17:45,NaT,NaN,NaN,2015.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447,1357,3703.0,FF4A7306XPvAibaEXe9CNoSsNIodZxiUJqcIH2Wu,2020-12-07,bangcap/18520499/18520499_bangcap_TOEIC_LR_202...,TOEIC_LR,240.0,128.0,225.0,113.0,465,NaN,1,NaN,NaT,NaN,NaN,2018.0,5.0
1448,1263,3524.0,FF549C85XPvAibaEXe8OSOsXgIr1IUHIn9A/2WBl,2020-11-12,bangcap/19520147/19520147_bangcap_TOEIC_LR_202...,TOEIC_LR,250.0,144.0,225.0,148.0,475,NaN,1,NaN,NaT,NaN,NaN,2019.0,3.0
1449,481,731.0,FF80CE6FXPvAibaEXe9spJK2tF1F77jXwcbLtAoG,2019-07-31,bangcap/18521111/18521111_bangcap_TOEIC_LR_201...,TOEIC_LR,245.0,144.0,290.0,148.0,NaN,NaN,1,2019-10-31 16:12:04,NaT,NaN,NaN,2018.0,3.0
1450,44,78.0,FFA60AE3XPvAibaEXe8Vh+AQMuj+NNj47wSDJ4ja,NaN,https://qldt.uit.edu.vn/sites/default/files/si...,TOEIC_LR,485.0,143.0,425.0,143.0,NaN,Miễn Anh văn 3,1,2018-02-05 10:36:54,NaT,NaN,NaN,2017.0,1.0


In [27]:
df = df.merge(cc[['mssv','loaixn','hockythu','listening','speaking','reading','writing','tongdiem']], how='left', on=['mssv','hockythu'])
df[255:260]

,mssv,hocky,namhoc,dtbhk,sotchk,lopsh,hedaotao,namnhaphoc,hockythu,diemtb,loaixn,listening,speaking,reading,writing,tongdiem
255,00ED4740XPvAibaEXe8t4cDFHKW3STPGGpmHJW7E,2,2019,6.89,12,MTCL2018,1,2018.0,4.0,5.817500,NaN,NaN,NaN,NaN,NaN,NaN
256,00ED4740XPvAibaEXe8t4cDFHKW3STPGGpmHJW7E,1,2020,5.96,18,MTCL2018,1,2018.0,5.0,5.846000,TOEIC_LR,310.0,128.0,290.0,113.0,600
257,00F2C35AXPvAibaEXe9p78qxnH96y4B0n45LtsTr,1,2015,3.53,13,MMTT2015,0,2015.0,1.0,3.530000,NaN,NaN,NaN,NaN,NaN,NaN
258,00F2C35AXPvAibaEXe9p78qxnH96y4B0n45LtsTr,2,2015,3.56,18,MMTT2015,0,2015.0,2.0,3.545000,NaN,NaN,NaN,NaN,NaN,NaN
259,00F2C35AXPvAibaEXe9p78qxnH96y4B0n45LtsTr,1,2016,6.06,25,MMTT2015,0,2015.0,3.0,4.383333,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
lr_de = 400
sw_de = 160
l_imputer = SimpleImputer(strategy='constant', fill_value=int(lr_de/2))
s_imputer = SimpleImputer(strategy='constant', fill_value=int(sw_de/2))
r_imputer = SimpleImputer(strategy='constant', fill_value=int(lr_de/2))
w_imputer = SimpleImputer(strategy='constant', fill_value=int(sw_de/2))

l_imputer.fit(df[['listening']])
s_imputer.fit(df[['speaking']])
r_imputer.fit(df[['reading']])
w_imputer.fit(df[['writing']])

SimpleImputer(fill_value=80, strategy='constant')

In [29]:
df['listening'] = l_imputer.transform(df[['listening']]).reshape(1,-1)[0]
df['reading'] = r_imputer.transform(df[['reading']]).reshape(1,-1)[0]
df['writing'] = w_imputer.transform(df[['writing']]).reshape(1,-1)[0]
df['speaking'] = s_imputer.transform(df[['speaking']]).reshape(1,-1)[0]

#### diemtadauvao
Điểm thi tiếng anh đầu vào

In [31]:
dv = pd.read_excel('xeploaiav.xlsx')
dv.columns = ['id', 'mssv', 'listening', 'reading', 'avdauvao', 'mamh', 'ghichu']
dv[3377:3382]

,id,mssv,listening,reading,avdauvao,mamh,ghichu
3377,3418.0,E44D1E6CXPvAibaEXe/k62DFAfrQTsS8tHO2IoFl,32,27,59.0,AVSC2,NULL
3378,3419.0,C0C9C20EXPvAibaEXe/y2t7DVG8TDRg2y/nF6frs,39,33,72.0,ENG02,NULL
3379,3420.0,4784C672XPvAibaEXe9Tdz4CswjA16eg3cXIH7xa,47,69,630.0,ENG05,Qui đổi TOEIC
3380,3421.0,280F7F2EXPvAibaEXe/TOGeH6x1BlQUztAN4ABTM,45,68,610.0,Miễn ENG03,Qui đổi TOEIC
3381,3422.0,3A16B235XPvAibaEXe8NM1/5chXJmoMJkfAtL/mC,46,67,610.0,Miễn ENG03,Qui đổi TOEIC


In [32]:
dv[dv.mamh == ' AVSC1'].avdauvao.describe()

,avdauvao
count,1261.000000
mean,102.406820
std,83.270368
min,6.000000
25%,38.000000
50%,44.000000
75%,195.000000
max,265.000000


Chuyển đổi thang điểm Toeic đầu vào thành điểm thi đầu vào

In [33]:
for i in range(len(dv)):
  try:
    if ('TOEIC' in dv['ghichu'][i]) or (dv.avdauvao[i] > 130):
      if dv.avdauvao[i] < 250: dv.avdauvao[i] = 38
      elif dv.avdauvao[i] < 300: dv.avdauvao[i] = 52
      elif dv.avdauvao[i] < 350: dv.avdauvao[i] = 57
      elif dv.avdauvao[i] < 400: dv.avdauvao[i] = 75
      elif dv.avdauvao[i] < 450: dv.avdauvao[i] = 82
      elif dv.avdauvao[i] < 500: dv.avdauvao[i] = 91
      else: dv.avdauvao[i] = 101
  except:
    1
dv[3377:3382]

,id,mssv,listening,reading,avdauvao,mamh,ghichu
3377,3418.0,E44D1E6CXPvAibaEXe/k62DFAfrQTsS8tHO2IoFl,32,27,59.0,AVSC2,NULL
3378,3419.0,C0C9C20EXPvAibaEXe/y2t7DVG8TDRg2y/nF6frs,39,33,72.0,ENG02,NULL
3379,3420.0,4784C672XPvAibaEXe9Tdz4CswjA16eg3cXIH7xa,47,69,101.0,ENG05,Qui đổi TOEIC
3380,3421.0,280F7F2EXPvAibaEXe/TOGeH6x1BlQUztAN4ABTM,45,68,101.0,Miễn ENG03,Qui đổi TOEIC
3381,3422.0,3A16B235XPvAibaEXe8NM1/5chXJmoMJkfAtL/mC,46,67,101.0,Miễn ENG03,Qui đổi TOEIC


In [34]:
df = df.merge(dv[['mssv','avdauvao']], how='left', on=['mssv'])
df[-5:]

,mssv,hocky,namhoc,dtbhk,sotchk,lopsh,hedaotao,namnhaphoc,hockythu,diemtb,loaixn,listening,speaking,reading,writing,tongdiem,avdauvao
50945,FFFA4234XPvAibaEXe+OhmjSj4XEzdTGgcTdT9fT,2,2017,7.86,19,ATTN2016,3,2016.0,4.0,7.427500,NaN,200.0,80.0,200.0,80.0,NaN,43.0
50946,FFFA4234XPvAibaEXe+OhmjSj4XEzdTGgcTdT9fT,1,2018,8.31,18,ATTN2016,3,2016.0,5.0,7.604000,NaN,200.0,80.0,200.0,80.0,NaN,43.0
50947,FFFA4234XPvAibaEXe+OhmjSj4XEzdTGgcTdT9fT,2,2018,8.49,19,ATTN2016,3,2016.0,6.0,7.751667,NaN,200.0,80.0,200.0,80.0,NaN,43.0
50948,FFFA4234XPvAibaEXe+OhmjSj4XEzdTGgcTdT9fT,1,2019,7.25,18,ATTN2016,3,2016.0,7.0,7.680000,NaN,200.0,80.0,200.0,80.0,NaN,43.0
50949,FFFA4234XPvAibaEXe+OhmjSj4XEzdTGgcTdT9fT,2,2019,8.10,13,ATTN2016,3,2016.0,8.0,7.732500,VNU-EPT,335.0,135.0,335.0,135.0,271,43.0


In [35]:
dauvao_imputer = SimpleImputer(strategy='mean', fill_value=tb(df['avdauvao']))
dauvao_imputer.fit(df[['avdauvao']])
df['avdauvao'] = dauvao_imputer.transform(df[['avdauvao']])

#### trinhdota
Derived feature, trình độ tiếng anh hiện tại theo chương trình học

In [37]:
diem = pd.read_excel('diem.xlsx')
diem.head()

,id,mssv,mamh,malop,sotc,namhoc,hocky,diem,trangthai,mamh_tt
0,137424.0,31D5D488XPvAibaEXe85Kg8gbEhwbxD0x3mi2el8,CS1113,CS1113.D11,4.0,2012.0,1.0,0.0,2.0,NULL
1,140865.0,31D5D488XPvAibaEXe85Kg8gbEhwbxD0x3mi2el8,PH001,PH001.D11,4.0,2012.0,1.0,0.0,1.0,NULL
2,141624.0,31D5D488XPvAibaEXe85Kg8gbEhwbxD0x3mi2el8,ENGL1113,ENGL1113.D11CTTT,3.0,2012.0,1.0,0.0,2.0,NULL
3,141651.0,31D5D488XPvAibaEXe85Kg8gbEhwbxD0x3mi2el8,ADENG1,ADENG1.D11CTTT,0.0,2012.0,1.0,0.0,2.0,NULL
4,141684.0,31D5D488XPvAibaEXe85Kg8gbEhwbxD0x3mi2el8,SS001,SS001.D11CTTT,5.0,2012.0,1.0,0.0,2.0,NULL


In [38]:
monta = [' ENBT', ' EN001', ' EN002', ' EN003',' EN004', ' EN005', ' EN006 ', ' ADENG1', ' ADENG2', ' ADENG3', ' ENGL1113', ' ENGL1213']
tamap = {' ENBT': 0,
         ' EN001': 1,
         ' EN002': 2,
         ' EN003': 3,
         ' EN004': 4,
         ' EN005': 5,
         ' EN006': 6,
         ' ADENG1': 7,
         ' ADENG2': 8,
         ' ADENG3': 9,
         ' ENGL1113': 10,
         ' ENGL1213': 11
         }
diem = diem[diem.mamh.isin(monta)]

In [39]:
diem2 = diem.merge(df[['mssv','namnhaphoc']].drop_duplicates(), how='inner', on=['mssv'])
diem2['hockythu'] = cotHocKyThu(diem2)
diem2 = diem2[diem2.hockythu > 0]
diem2.columns = ['id', 'mssv', 'trinhdota', 'malop', 'sotc', 'namhoc', 'hocky', 'diem', 'trangthai', 'mamh_tt', 'namnhaphoc', 'hockythu']
diem2.head()

,id,mssv,trinhdota,malop,sotc,namhoc,hocky,diem,trangthai,mamh_tt,namnhaphoc,hockythu
0,188976.0,599DFFB8XPvAibaEXe879+AOg1gh8lJvChSN7o+V,EN001,EN001.E11.ANTT,4.0,2013.0,1.0,6.0,1.0,NULL,2013.0,1.0
1,188977.0,51E1C3E0XPvAibaEXe+4hxKfaQWuhLp2zzmiiyRe,EN001,EN001.E11.ANTT,4.0,2013.0,1.0,6.5,1.0,NULL,2013.0,1.0
2,188978.0,2F237AA9XPvAibaEXe/YKAlYnC3m967dOM4WK2IJ,EN001,EN001.E11.ANTT,4.0,2013.0,1.0,7.0,1.0,NULL,2013.0,1.0
3,188979.0,6AEF2EC0XPvAibaEXe86736b6Ol+/EwtIJHvLVRJ,EN001,EN001.E11.ANTT,4.0,2013.0,1.0,9.0,1.0,NULL,2013.0,1.0
4,188980.0,6F75613CXPvAibaEXe8/3iNqySORbxrmFjIeDct7,EN001,EN001.E11.ANTT,4.0,2013.0,1.0,7.0,1.0,NULL,2013.0,1.0


In [40]:
df = df.merge(diem2[['mssv','trinhdota','hockythu']].drop_duplicates(), how='left', on=['mssv','hockythu'])
df.head()

,mssv,hocky,namhoc,dtbhk,sotchk,lopsh,hedaotao,namnhaphoc,hockythu,diemtb,loaixn,listening,speaking,reading,writing,tongdiem,avdauvao,trinhdota
0,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,1,2019,8.84,17,TMĐT2019,0,2019.0,1.0,8.840000,NaN,200.0,80.0,200.0,80.0,NaN,82.000000,NaN
1,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,2,2019,9.00,21,TMĐT2019,0,2019.0,2.0,8.920000,NaN,200.0,80.0,200.0,80.0,NaN,82.000000,NaN
2,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,1,2020,9.11,15,TMĐT2019,0,2019.0,3.0,8.983333,NaN,200.0,80.0,200.0,80.0,NaN,82.000000,NaN
3,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,1,2013,5.41,22,KTMT2013,0,2013.0,1.0,5.410000,NaN,200.0,80.0,200.0,80.0,NaN,52.554571,EN001
4,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,2,2013,7.07,15,KTMT2013,0,2013.0,2.0,6.240000,NaN,200.0,80.0,200.0,80.0,NaN,52.554571,EN002


##### fill trinhdota

In [41]:
df[df.hockythu == 1].trinhdota.value_counts()

,count
trinhdota,
EN001,1381
EN004,1380
ENBT,1000
EN005,208
EN002,146
ENGL1113,125
ADENG1,75
ADENG3,17
EN003,1


Chuyển đổi mã các môn học tương đương

In [42]:
conv = {
    ' AVSC': ' ENBT',
    ' AVSC1': ' EN001',
    ' AVSC2': ' EN002',
    ' ENG01': ' EN002',
    ' ENG02': ' EN003',
    ' ENG03': ' EN004',
    ' ENG04': ' EN005',
    ' ENG05': ' EN006',
    ' Miễn ENG03': ' EN005',
}

for i in tqdm(range(len(df))):
  if (df.hockythu[i] == 1) & (str(df.trinhdota[i]) == 'nan'):
    if (df.mssv[i] in list(dv.mssv)):
      mon = dv[dv.mssv == df.mssv[i]].mamh.values[0]
      if mon in conv:
        mon = conv[mon]
      df.trinhdota[i] = mon

100%|██████████| 51477/51477 [00:11<00:00, 4664.46it/s]


In [43]:
df.trinhdota.value_counts()

,count
trinhdota,
EN002,3601
EN001,2963
EN004,2126
EN003,2056
EN005,1317
ENBT,1293
ENGL1113,126
EN006,125
ENGL1213,123


Forward fill

In [44]:
for i in tqdm(range(len(df))):
  if (str(df.trinhdota[i]) in tamap) == False:
    j = i-1
    while df.mssv[i] == df.mssv[j]:
      if (str(df.trinhdota[j]) in tamap) == True:
        df.trinhdota[i] = df.trinhdota[j]
        break
      j -= 1
      if j < 0:
        break

df.trinhdota.value_counts()

100%|██████████| 51477/51477 [00:13<00:00, 3812.94it/s]


,count
trinhdota,
EN003,11368
EN002,11237
EN004,8229
EN005,7619
EN001,5654
ENBT,4314
ENGL1213,703
EN006,480
ADENG2,222


In [45]:
df = df[df.trinhdota.isin(tamap)].reset_index(drop=True)

## Label

class 1: tiến độ học của sinh viên hoàn thành tiếng anh đúng hạn


class 0: tiến độ học của sinh viên hoàn thành tiếng anh trễ hạn

#### Danh sách MSSV của các SV tốt nghiệp đúng hạn

In [47]:
tn = pd.read_excel('totnghiep.xlsx')
tn.head()

,id,mssv,xeploai,soquyetdinh,ngaycapvb
0,76328.0,E95E7C6DXPvAibaEXe+1j/AqdkpM22DHf6P99fDJ,Khá,178/QĐ_ĐHCNTT,14/04/2017
1,76663.0,D0FE4969XPvAibaEXe/yXiKgsgy0slCmJ5EKt6Ki,Giỏi,713/QĐ_ĐHCNTT,06/10/2017
2,76664.0,40F7E8D0XPvAibaEXe+nZBq3b0XEhfcwXLmoc4Pj,Giỏi,713/QĐ_ĐHCNTT,06/10/2017
3,76665.0,0A049F45XPvAibaEXe9CtFSNMbfIz9qE7i0Fu4My,Khá,713/QĐ_ĐHCNTT,06/10/2017
4,76666.0,7A7166DAXPvAibaEXe83V/kFvw2bBZ0o/KVHZc30,Khá,713/QĐ_ĐHCNTT,06/10/2017


In [48]:
tn = tn.merge(df[['mssv','namnhaphoc']].drop_duplicates(), how='inner', on=['mssv'])

In [49]:
tn.namnhaphoc.value_counts()

,count
namnhaphoc,
2013.0,549
2014.0,508
2015.0,459
2016.0,277
2017.0,37


In [50]:
format1 = '%d/%m/%Y'
format2 = '%Y-%m-%d'
tn.columns = ['id', 'mssv', ' xeploai', ' soquyetdinh', 'ngayxl', 'namnhaphoc']

tn['hockythu'] = hocKyThuCc(tn)
tn.head()

,id,mssv,xeploai,soquyetdinh,ngayxl,namnhaphoc,hockythu
0,76328.0,E95E7C6DXPvAibaEXe+1j/AqdkpM22DHf6P99fDJ,Khá,178/QĐ_ĐHCNTT,14/04/2017,2013.0,8.0
1,76663.0,D0FE4969XPvAibaEXe/yXiKgsgy0slCmJ5EKt6Ki,Giỏi,713/QĐ_ĐHCNTT,06/10/2017,2013.0,9.0
2,76664.0,40F7E8D0XPvAibaEXe+nZBq3b0XEhfcwXLmoc4Pj,Giỏi,713/QĐ_ĐHCNTT,06/10/2017,2013.0,9.0
3,76665.0,0A049F45XPvAibaEXe9CtFSNMbfIz9qE7i0Fu4My,Khá,713/QĐ_ĐHCNTT,06/10/2017,2013.0,9.0
4,76666.0,7A7166DAXPvAibaEXe83V/kFvw2bBZ0o/KVHZc30,Khá,713/QĐ_ĐHCNTT,06/10/2017,2013.0,9.0


In [51]:
tn.hockythu.value_counts()

,count
hockythu,
10.0,674
9.0,450
11.0,327
12.0,185
13.0,127
8.0,42
7.0,24
1.0,1


In [52]:
tn = tn[tn.hockythu > 1]

In [53]:
dunghan1 = tn[tn.hockythu < 10].mssv.tolist()

#### Danh sách MSSV của các sinh viên nộp chứng chỉ tiếng anh đúng hạn

In [54]:
dunghan2 = []

for i in range(len(cc)):
  if ((cc['listening'][i] + cc['reading'][i] >= 450) & (cc['writing'][i] + cc['speaking'][i] >= 185)):
    dunghan2.append(cc['mssv'][i])

In [55]:
dunghan = list(set(dunghan1 + dunghan2))

#### Danh sách MSSV của các SV tốt nghiệp trễ hạn

In [56]:
trehan1 = df[df.hockythu > 9].mssv.drop_duplicates().tolist()

In [57]:
trehan = list(set(trehan1) - set(dunghan2))

In [59]:
y = []
for i in range(len(df)):
  if df.mssv[i] in trehan:
    y.append(0)
  elif df.mssv[i] in dunghan:
    y.append(1)
  else:
    y.append(2)

df['tadunghan'] = y

In [60]:
fdf = df[df.tadunghan != 2].reset_index(drop=True)
fdf = fdf[fdf.trinhdota.isin(tamap)]
fdf

,mssv,hocky,namhoc,dtbhk,sotchk,lopsh,hedaotao,namnhaphoc,hockythu,diemtb,loaixn,listening,speaking,reading,writing,tongdiem,avdauvao,trinhdota,tadunghan
0,005D6433XPvAibaEXe9JJ0DEqozv6JkD0BJUpbna,1,2016,8.02,17,KTMT2016,0,2016.0,1.0,8.020000,NaN,200.0,80.0,200.0,80.0,NaN,37.0,EN004,1
1,005D6433XPvAibaEXe9JJ0DEqozv6JkD0BJUpbna,2,2016,6.92,21,ANTN2016,0,2016.0,2.0,7.470000,NaN,200.0,80.0,200.0,80.0,NaN,37.0,EN005,1
2,005D6433XPvAibaEXe9JJ0DEqozv6JkD0BJUpbna,1,2017,8.55,22,ATTN2016,3,2016.0,3.0,7.830000,NaN,200.0,80.0,200.0,80.0,NaN,37.0,EN005,1
3,005D6433XPvAibaEXe9JJ0DEqozv6JkD0BJUpbna,2,2017,7.32,23,ATTN2016,3,2016.0,4.0,7.702500,NaN,200.0,80.0,200.0,80.0,NaN,37.0,EN005,1
4,005D6433XPvAibaEXe9JJ0DEqozv6JkD0BJUpbna,1,2018,8.25,15,ATTN2016,3,2016.0,5.0,7.812000,NaN,200.0,80.0,200.0,80.0,NaN,37.0,EN005,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17405,FFFA4234XPvAibaEXe+OhmjSj4XEzdTGgcTdT9fT,2,2017,7.86,19,ATTN2016,3,2016.0,4.0,7.427500,NaN,200.0,80.0,200.0,80.0,NaN,43.0,EN004,1
17406,FFFA4234XPvAibaEXe+OhmjSj4XEzdTGgcTdT9fT,1,2018,8.31,18,ATTN2016,3,2016.0,5.0,7.604000,NaN,200.0,80.0,200.0,80.0,NaN,43.0,EN004,1
17407,FFFA4234XPvAibaEXe+OhmjSj4XEzdTGgcTdT9fT,2,2018,8.49,19,ATTN2016,3,2016.0,6.0,7.751667,NaN,200.0,80.0,200.0,80.0,NaN,43.0,EN004,1
17408,FFFA4234XPvAibaEXe+OhmjSj4XEzdTGgcTdT9fT,1,2019,7.25,18,ATTN2016,3,2016.0,7.0,7.680000,NaN,200.0,80.0,200.0,80.0,NaN,43.0,EN004,1


In [61]:
fdf.tadunghan.value_counts()

,count
tadunghan,
1,11164
0,6246


Chia tập dữ liệu train, validation, test

In [62]:
d = fdf[['hedaotao','hockythu','diemtb','listening','speaking','reading','writing','avdauvao','trinhdota']]
d['trinhdota'] = d['trinhdota'].map(lambda x: tamap[x])
y = fdf['tadunghan']

no = StandardScaler().fit_transform(d)

X_train, X_test, y_train, y_test = train_test_split(no, y, test_size=0.3, random_state=90)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=90)

# Training mô hình

### Grid search tìm bộ siêu tham số tốt nhất cho mô hình

###### Logistic regression

In [63]:
grid_logreg = {'C': np.logspace(-3,3,7), 'penalty': ["l1","l2"]}

grid = GridSearchCV(LogisticRegression(), grid_logreg, cv=5, verbose=1, n_jobs=-1)
grid.fit(X_val, y_val)

Fitting 5 folds for each of 14 candidates, totalling 70 fits


GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2']},
             verbose=1)

In [64]:
grid.best_params_

{'C': 0.1, 'penalty': 'l2'}

###### Random forest

In [65]:
grid_rf = {
    'max_features': ['sqrt', 'log2'],
    'min_samples_leaf': [1, 3],
    'min_samples_split': [2, 8],
    'n_estimators': [10, 100, 200]
}

grid = GridSearchCV(RandomForestClassifier(), grid_rf, cv=5, verbose=4, n_jobs=-1)
grid.fit(X_val, y_val)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 3],
                         'min_samples_split': [2, 8],
                         'n_estimators': [10, 100, 200]},
             verbose=4)

In [66]:
grid.best_params_

{'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 200}

###### Support vector machine

In [67]:
grid_svm = {'C': [0.1, 1, 10],
              'gamma': [1, 0.1, 0.01],
              'kernel': ['rbf', 'linear']}

grid = GridSearchCV(SVC(), grid_svm, cv=5, verbose=5, n_jobs=-1)
grid.fit(X_val, y_val)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'gamma': [1, 0.1, 0.01],
                         'kernel': ['rbf', 'linear']},
             verbose=5)

In [68]:
grid.best_params_

{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}

### Huấn luyện mô hình

In [69]:
lr = LogisticRegression(C=0.1, penalty='l2')
lr.fit(X_train, y_train)

with open('lr.pkl', 'wb') as f:
  pickle.dump(lr, f)

In [70]:
rf = RandomForestClassifier(max_features='sqrt', min_samples_leaf=3, min_samples_split=2, n_estimators=200)
rf.fit(X_train, y_train)

with open('rf.pkl', 'wb') as f:
  pickle.dump(rf, f)

In [71]:
sv = SVC(C=10, gamma=0.1, kernel='rbf')
sv.fit(X_train, y_train)

with open('sv.pkl', 'wb') as f:
  pickle.dump(sv, f)

# Đánh giá

In [72]:
y_pred_lr = lr.predict(X_test)
print(classification_report(y_test, y_pred_lr))

              precision    recall  f1-score   support

           0       0.75      0.66      0.70       953
           1       0.82      0.87      0.84      1658

    accuracy                           0.79      2611
   macro avg       0.78      0.77      0.77      2611
weighted avg       0.79      0.79      0.79      2611



In [73]:
y_pred_rf = rf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75       953
           1       0.86      0.85      0.86      1658

    accuracy                           0.82      2611
   macro avg       0.80      0.80      0.80      2611
weighted avg       0.82      0.82      0.82      2611



In [74]:
y_pred_rf = lr.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.75      0.66      0.70       953
           1       0.82      0.87      0.84      1658

    accuracy                           0.79      2611
   macro avg       0.78      0.77      0.77      2611
weighted avg       0.79      0.79      0.79      2611

